In [1]:
import os
import pandas as pd
import numpy as np
import json
import requests
import zipfile
import datetime
from tqdm import tqdm
root_path = os.path.join("event")
result_path = os.path.join("event_drug_symptom")

In [2]:
to_year = {
            '800': 10,          # decade
            '801': 1,           # year
            '802': 1 / 12,      # month
            '803': 1 / (12 * 4),# week
            '804': 1 / 365,     # days
            '805': 1 / 8760,    # hour
        }
decode_sex = {0: 'unknown', 1: "Male", 2: "Female"}

decode_characterization = {
            1: "Suspect (the drug was considered by the reporter to be the cause)",
            2: "Concomitant (the drug was reported as being taken along with the suspect drug)",
            3: "Interacting (the drug was considered by the reporter to have interacted with the suspect drug)"
        }
decode_serious = {
            1: "The adverse event resulted in death, a life threatening condition, hospitalization, disability, \
            congenital anomaly, or other serious condition",
            2: "The adverse event did not result in any of the above"}
decode_death = {
            0: "survive",
            1: "death"
        }
decode_sender = {
            1: "Physician",
            2: "Pharmacist",
            3: "Other health professional",
            4: "Lawyer",
            5: "Consumer or non-health professional",
            0: "unknown"
        }

In [3]:
# from download.py with modification
def download(arg):
    """download file at url and save at dir """
    url, directory = arg
    r = requests.get(url, allow_redirects=True)
    open(directory, 'wb').write(r.content)
    with zipfile.ZipFile(directory, 'r') as zip_ref:
        zip_ref.extractall(os.path.dirname(directory))
    os.remove(directory)

def get_event_links(path):
    """obtain all download links for events from fdc's provided download json"""
    with open(path, 'r') as handle:
        file = json.load(handle)
    events = file['results']["drug"]["event"]
    links = []
    for p in events['partitions']:
        links.append(p['file'])
    return links

def get_ndc_links(path):
    """obtain all download links for events from fdc's provided download json"""
    with open(path, 'r') as handle:
        file = json.load(handle)
    events = file['results']["drug"]["ndc"]
    links = []
    for p in events['partitions']:
        links.append(p['file'])
    return links

links = get_event_links("download.json")
ndc_links = get_ndc_links("download.json")

In [4]:
# Download ndc file
url = ndc_links[0]
infos = url.split("/")[3:] # getting rid of https and website
path = ''
for i in infos:
    path = os.path.join(path, i)
    if i.endswith('.zip'):
        break
    else:
        if not os.path.exists(path):
            os.mkdir(path)
        path = path.strip("\n")
        url = url.strip("\n")
download([url, path])

In [5]:
"""Create a symptom database from symptoms.json"""

symptoms = json.load(open("symptoms.json", "r"))
data = {"id":[], "symptom":[]}
for i, ele in tqdm(enumerate(symptoms)):
    data["id"].append(i)
    data["symptom"].append(ele)
df = pd.DataFrame(data)
df.to_csv(os.path.join("drug","symptoms_dataset.csv"), index=False)

21099it [00:00, 2350731.02it/s]


In [6]:
"""Create datasets for ndc's drug listings and their active ingredients""" 
"""id is associated with the brand_name"""

ndc_data = {"id":[], "brand_name":[], "generic_name":[], "product_type":[], "manufacturer_name":[]}
components = {"id":[], "active_ingredient":[]}

encode_product_type = {"HUMAN OTC DRUG":0, 
                       "HUMAN PRESCRIPTION DRUG":1, 
                       "PLASMA DERIVATIVE":2, 
                       "VACCINE":3, 
                       "BULK INGREDIENT":4, 
                       "DRUG FOR FURTHER PROCESSING":5, 
                       "CELLULAR THERAPY":6, 
                       "LICENSED VACCINE BULK INTERMEDIATE":7,
                       "STANDARDIZED ALLERGENIC":8, "NON-STANDARDIZED ALLERGENIC":9}

with open(os.path.join("ndc", "drug-ndc-0001-of-0001.json")) as ifs:
    ndc = json.load(ifs)
    ndc = ndc["results"] # list of dict
    for i, ele in tqdm(enumerate(ndc)):
        if "generic_name" in ele:
            for name in ele["generic_name"].split(","):
                ndc_data["id"].append(i)
                ndc_data["generic_name"].append(name.strip())
                ndc_data["brand_name"].append(ele["brand_name_base"])
                ndc_data["product_type"].append(encode_product_type[ele["product_type"]])
                if "manufacturer_name" in ele["openfda"]:
                    ndc_data["manufacturer_name"].append(ele["openfda"]["manufacturer_name"][0])
                else:
                    ndc_data["manufacturer_name"].append("__NULL__")
        else:
            ndc_data["id"].append(i)
            ndc_data["generic_name"].append("__NULL__")
            ndc_data["brand_name"].append(ele["brand_name_base"])
            ndc_data["product_type"].append(encode_product_type[ele["product_type"]])
            if "manufacturer_name" in ele["openfda"]:
                ndc_data["manufacturer_name"].append(ele["openfda"]["manufacturer_name"][0])
            else:
                ndc_data["manufacturer_name"].append("__NULL__")
        if "active_ingredients" in ele:
            for comp in ele["active_ingredients"]:
                components["id"].append(i)
                components["active_ingredient"].append(comp["name"])
    ndc_df = pd.DataFrame(ndc_data)
    ndc_df.to_csv(os.path.join("drug","drug_dataset.csv"), index=False)
    comp_df = pd.DataFrame(components)
    comp_df.to_csv(os.path.join("drug","active_ingredient_dataset.csv"), index=False)

126701it [00:00, 456978.05it/s]


In [7]:
"""Drug and Symptom Reverse Lookup"""

#Brand_Name -> ID
drug_vocab = {}
drug_df = pd.read_csv(os.path.join("drug","drug_dataset.csv"))
for i,row in tqdm(drug_df.iterrows()):
    drug_vocab[row["brand_name"]] = row["id"]

# Keep track of discovered drugs
num_drugs = len(drug_df)
missing_vocab = {}
missing_drugs = {"id":[], "brand_name":[]}

# Symptom -> ID
symptom_vocab = {}
symptom_df = pd.read_csv(os.path.join("drug","symptoms_dataset.csv"))
symptom_df['symptom'] = symptom_df['symptom'].str.lower()
for i,row in tqdm(symptom_df.iterrows()):
    symptom_vocab[row["symptom"]] = row["id"]
    
event_id = 0

205412it [00:12, 16003.20it/s]
21099it [00:01, 16038.98it/s]


In [8]:
if not os.path.exists(os.path.join("drug","events")):
    os.mkdir(os.path.join("drug","events"))
    os.mkdir(os.path.join("drug","drugs"))
    os.mkdir(os.path.join("drug","characteristics"))
    os.mkdir(os.path.join("drug","symptoms"))

In [9]:
# Drug Dataset:
    # ID (event)
    # postings list based off drug_dataset
    # missing list based off missing_dataset
# Characterization dataset:
    # ID (event)
    # postings list based off drug_dataset
    # missing list based off missing_dataset
# Symptoms dataset:
    # ID (event)
    # postings list based off symptoms_dataset

for link_index, url in tqdm(enumerate(links)):
    # Download an event file
    infos = url.split("/")[3:] # getting rid of https and website
    path = ''
    for i in infos:
        path = os.path.join(path, i)
        if i.endswith('.zip'):
            break
        else:
            if not os.path.exists(path):
                os.mkdir(path)
            path = path.strip("\n")
            url = url.strip("\n")
    download([url, path])

    folder_name = path.split("\\")[-2]
    file_name = path.split(".")[0]
    split_name = file_name.split("\\")[-1]

    # Grab and decode events
    events = json.load(open(file_name+".json", "r"))
    events = events["results"]

    # Event Dataset:
    event_data = {"ID":[], "sex":[], "age":[], "serious":[], "death":[], "sender":[], "time":[]}
    drug_data = {"ID":[], "postings":[], "missing":[]}
    char_data = {"ID":[], "postings":[], "missing":[]}
    symp_data = {"ID":[], "postings":[]}

    for event in events:
        patient = event["patient"]

        # Event
        event_data["ID"].append(event_id)
        event_data["serious"].append(event["serious"])

        if "patientsex" in patient:
            event_data["sex"].append(decode_sex[int(patient['patientsex'])])
        else:
            event_data["sex"].append(0)
        try:
            event_data["age"].append(int(patient["patientonsetage"]) * to_year[patient["patientonsetageunit"]])
        except:
            event_data["age"].append(-1)
        if "seriousnessdeath" in event:
            event_data["death"].append(1)
        else:
            event_data["death"].append(0)
        try:
            event_data["sender"].append(decode_sender[int(event["primarysource"]["qualification"])])
        except:
            event_data["sender"].append(0)
        try:
            time = event['receiptdate']
            timestamp = datetime.datetime(int(time[0:4]), int(time[4:6]), int(time[6::]), 0, 0).timestamp()
            event_data["time"].append(timestamp)
        except:
            event_data["time"].append(-1)

        # Drugs & Characteristics
        drug_data["ID"].append(event_id)
        char_data["ID"].append(event_id)
        d = []
        c = []
        m_d = []
        m_c = []
        for ele in patient["drug"]:
            if "medicinalproduct" in ele:
                # Check if drug in ndc data
                if ele["medicinalproduct"] in drug_vocab:
                    d.append(drug_vocab[ele["medicinalproduct"]])
                    if "drugcharacterization" in ele: 
                        c.append(ele["drugcharacterization"])
                    else:
                        c.append(0)
                else:
                    # Check if this missing drug has been seen before
                    if ele["medicinalproduct"] not in missing_vocab:
                        missing_drugs["id"].append(num_drugs)
                        missing_drugs["brand_name"].append(ele["medicinalproduct"])
                        missing_vocab[ele["medicinalproduct"]] = num_drugs
                        num_drugs += 1
                    m_d.append(missing_vocab[ele["medicinalproduct"]])                        
                    if "drugcharacterization" in ele:
                        m_c.append(ele["drugcharacterization"])
                    else:
                        m_c.append(0)
                
        drug_data["postings"].append(" ".join([str(x) for x in d]))
        char_data["postings"].append(" ".join([str(x) for x in c]))
        drug_data["missing"].append(" ".join([str(x) for x in m_d]))
        char_data["missing"].append(" ".join([str(x) for x in m_c]))

        # Symptoms
        symp_data["ID"].append(event_id)
        if "reaction" in patient:
            s = []
            for sym in patient["reaction"]:
                if "reactionmeddrapt" in sym:
                    s.append(str(symptom_vocab[sym['reactionmeddrapt'].lower()]))
            symp_data["postings"].append(" ".join(s))
        else:
            symp_data["postings"].append("")

        event_id += 1

    # Remove download
    os.remove(file_name+".json")
    os.rmdir(os.path.join("\\".join(path.split("\\")[0:3])))

    # Write results to file
    if not os.path.exists(os.path.join("drug","events", folder_name)):
        os.mkdir(os.path.join("drug","events", folder_name))
        os.mkdir(os.path.join("drug","drugs", folder_name))
        os.mkdir(os.path.join("drug","characteristics", folder_name))
        os.mkdir(os.path.join("drug","symptoms", folder_name))

    event_df = pd.DataFrame(event_data)
    event_df.to_csv(os.path.join("drug", "events", folder_name, split_name+".csv"), index=False)

    drug_df = pd.DataFrame(drug_data)
    drug_df.to_csv(os.path.join("drug", "drugs", folder_name, split_name+".csv"), index=False)

    char_df = pd.DataFrame(char_data)
    char_df.to_csv(os.path.join("drug", "characteristics", folder_name, split_name+".csv"), index=False)

    symp_df = pd.DataFrame(symp_data)
    symp_df.to_csv(os.path.join("drug", "symptoms", folder_name, split_name+".csv"), index=False)

missing_df = pd.DataFrame(missing_drugs)
missing_df.to_csv(os.path.join("drug","missing_drugs_dataset.csv"), index=False)

900it [1:56:24, 19.73s/it]


In [10]:
print("num missing:", len(missing_drugs["brand_name"]))
print("num events:", event_id)

num missing: 614669
num events: 10432358


In [17]:
df = None
check = False
for folder in tqdm(os.listdir(os.path.join("drug", "drugs"))):
    for file in os.listdir(os.path.join("drug", "drugs", folder)):
        if check:
            df = pd.concat([df, pd.read_csv(os.path.join("drug", "drugs", folder, file))])
        else:
            df = pd.read_csv(os.path.join("drug", "drugs", folder, file))
            check = True

100%|██████████████████████████████████████████████████████████████████████████████████| 62/62 [01:57<00:00,  5.30s/it]


In [33]:
df = df.fillna("")
subset = df.loc[df["missing"]==""]
subset = subset.loc[subset["postings"]!=""]
subset = subset[["ID","postings"]]
print(len(subset))
subset.to_csv(os.path.join("drug","no_missing_drugs.csv"), index=False)

2360690


In [50]:
df2 = df
df2 = df2.fillna("")
for i, row in tqdm(df2.iterrows()):
    df2.iloc[i]["drug_count"] = len(row["postings"].split(" "))
    df2.iloc[i]["missing_count"] = len(row["missing"].split(" "))

0it [00:00, ?it/s]C:\Users\sjray\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\sjray\Anaconda3\lib\site-packages\pandas\core\series.py:1015: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value
C:\Users\sjray\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
6it [00:10,  1.72s/it]

KeyboardInterrupt: 

In [47]:
print(df2["drug_count"].mean())
print(df2["missing_count"].mean())
print(df2["drug_count"].head(10))

10432358.0
10432358.0
0    10432358
1    10432358
2    10432358
3    10432358
4    10432358
5    10432358
6    10432358
7    10432358
8    10432358
9    10432358
Name: drug_count, dtype: int64


In [63]:
ms = pd.read_csv(os.path.join("drug","missing_drugs_dataset.csv"))
miss = ms["brand_name"].values
miss = [str(x).lower().split("(")[0].strip() for x in miss]
print(len(miss))
test = list(set(miss))
print(len(test))
test = [x for x in test if "unknown" not in x]
print(len(test))

614669
447739
443797
